### 1. Data Ingestion

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
file_path=os.path.join(os.getcwd(), "data", "sample.pdf")

In [3]:
loader=PyPDFLoader(file_path)

In [4]:
documents=loader.load()

In [5]:
len(documents)

77

### this is a experimental thing there is no deterministic way to split the text

In [6]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

In [7]:
docs=text_splitter.split_documents(documents)

In [8]:
len(docs)

765

In [9]:
docs[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2023-07-20T00:30:36+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2023-07-20T00:30:36+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': 'e:\\KrishNaik LLMOps\\Document_Portal\\notebook\\data\\sample.pdf',
 'total_pages': 77,
 'page': 0,
 'page_label': '1'}

In [10]:
from langchain.vectorstores import FAISS

In [11]:
docs[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev'

In [14]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [15]:
load_dotenv()

True

In [27]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [28]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [29]:
doc_vector=embedding_model.embed_documents(docs[0].page_content)

In [30]:
len(embedding_model.embed_documents(docs[0].page_content)[0])

768

In [31]:
vectorstore=FAISS.from_documents(docs, embedding_model)

### This is a Retrieval proceesss

In [32]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.")

In [34]:
relevant_doc[0].page_content

'13B 18.9 66.1 52.6 62.3 10.9 46.9 37.0 33.9\n33B 26.0 70.0 58.4 67.6 21.4 57.8 39.8 41.7\n65B 30.7 70.7 60.5 68.6 30.8 63.4 43.5 47.6\nLlama 2\n7B 16.8 63.9 48.9 61.3 14.6 45.3 32.6 29.3\n13B 24.5 66.9 55.4 65.8 28.7 54.8 39.4 39.1\n34B 27.8 69.9 58.7 68.0 24.2 62.6 44.1 43.4\n70B 37.5 71.9 63.6 69.4 35.2 68.9 51.2 54.2\nTable 3: Overall performance on grouped academic benchmarks compared to open-source base models.'

In [39]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=1)

In [40]:
relevant_doc

[Document(id='2b38bf40-422c-4425-baa3-1fc5c8c1aca7', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'e:\\KrishNaik LLMOps\\Document_Portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 7, 'page_label': '8'}, page_content='13B 18.9 66.1 52.6 62.3 10.9 46.9 37.0 33.9\n33B 26.0 70.0 58.4 67.6 21.4 57.8 39.8 41.7\n65B 30.7 70.7 60.5 68.6 30.8 63.4 43.5 47.6\nLlama 2\n7B 16.8 63.9 48.9 61.3 14.6 45.3 32.6 29.3\n13B 24.5 66.9 55.4 65.8 28.7 54.8 39.4 39.1\n34B 27.8 69.9 58.7 68.0 24.2 62.6 44.1 43.4\n70B 37.5 71.9 63.6 69.4 35.2 68.9 51.2 54.2\nTable 3: Overall performance on grouped academic benchmarks compared to open-source base models.')]

In [41]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [42]:
retriever.invoke("llama2 finetuning benchmark experiments.")

[Document(id='2b38bf40-422c-4425-baa3-1fc5c8c1aca7', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'e:\\KrishNaik LLMOps\\Document_Portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 7, 'page_label': '8'}, page_content='13B 18.9 66.1 52.6 62.3 10.9 46.9 37.0 33.9\n33B 26.0 70.0 58.4 67.6 21.4 57.8 39.8 41.7\n65B 30.7 70.7 60.5 68.6 30.8 63.4 43.5 47.6\nLlama 2\n7B 16.8 63.9 48.9 61.3 14.6 45.3 32.6 29.3\n13B 24.5 66.9 55.4 65.8 28.7 54.8 39.4 39.1\n34B 27.8 69.9 58.7 68.0 24.2 62.6 44.1 43.4\n70B 37.5 71.9 63.6 69.4 35.2 68.9 51.2 54.2\nTable 3: Overall performance on grouped academic benchmarks compared to open-source base models.'),
 Document(id='50396f71-12ca-449d-b

### Question: user question
### Context: based on the question retrieving the info from the vector database

In [43]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [44]:
from langchain.prompts import PromptTemplate

In [45]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [46]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n        Answer the question based on the context provided below. \n        If the context does not contain sufficient information, respond with: \n        "I do not have enough information about this."\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:')

In [47]:
from langchain_core.output_parsers import StrOutputParser

In [48]:
parser=StrOutputParser()

In [49]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [50]:
from langchain_core.runnables import RunnablePassthrough

In [51]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

'<think>\nOkay, so I need to figure out the answer to the question: "Tell me about the llama2 finetuning benchmark experiments?" based on the provided context. Let me read through the context carefully.\n\nFirst, I notice there are several tables and sections discussing different models, including Llama 2. The context mentions "Fine-tuned" models like ChatGPT, MPT-instruct, Falcon-instruct, and Llama 2-Chat. There are numbers and percentages listed, but I\'m not entirely sure what each column represents. \n\nLooking at the tables, I see metrics such as "true", "info", and percentages. For example, under "Fine-tuned", ChatGPT has 78.46, 79.92, and 98.53. Llama 2-Chat 7B has 57.04, 60.59, and 96.45. These could be accuracy scores or some performance metrics across different benchmarks.\n\nIn the context, there\'s a mention of "safety benchmarks" in Section 4.1, but I don\'t have that section here. The tables provided seem to summarize overall performance across grouped academic benchmark

In [53]:
rag_chain.invoke("can you tell me Scaling trends for the reward model?")

"<think>\nOkay, so I need to figure out the scaling trends for the reward model based on the context provided. Let me read through the context again to find any relevant information.\n\nLooking at the context, I see a mention of Figure 6, which reports trends showing that larger models achieve higher performance with a similar volume of data. That's a positive sign. It also says that the scaling performance hasn't plateaued yet, which means there's still room for improvement as more data is annotated. \n\nAdditionally, the context talks about the impact of the margin-based loss on reward score distributions. Figure 27 shows that introducing a margin creates a binary split in the reward scores, especially with a larger margin. This suggests that the model's reward distribution becomes more distinct, which could imply better performance as the model scales.\n\nThere's also a section on the training objectives where they use a binary ranking loss. They mention that the reward models are w